In [23]:
from DLTools.ModelWrapper import ModelWrapper
from keras.layers import Input, Dense, Layer,BatchNormalization,Dropout,Flatten,pooling,Activation
import keras.models 
from keras.layers.convolutional import Conv3D
from keras.models import Sequential

In [70]:
class Conv_Sequential(ModelWrapper):
    def __init__(self,Name,
                 InputShape=(None, 4),
                 Depth = 0,
                 Widths = 0,
                 Activation = 'relu',
                 layers = None,
                 Loss = "mean_squared_logarithmic_error",
                 Optimizer = 'SGD',
                 Metrics = 'accuracy'
                  ):
        super(Conv_Sequential,self).__init__(Name,Loss,Optimizer)
        
        self.InputShape = self.MetaData["InputShape"] = InputShape
        self.Depth = self.MetaData["Depth"]= Depth
        self.Widths = self.MetaData["Widths"] = Widths
        self.Activation = self.MetaData["Activation"] = Activation
 
 #Creates a model based on sequential from Keras, and adds layers based on widths defined 
    def Build(self):
        model = Sequential()
        model.add(Activation('relu',input_shape=self.InputShape))
        #Cycles through the list of widths for the model and adds layers
        for i in xrange(0,self.Depth):
            model.add(Dense(self.Widths,input_shape = self.InputShape))
        #model.add(Activation('relu'))        
        #Downscales from the input shape to a size of (100,100,100)
        model.add(pooling.MaxPooling3D(pool_size=(2, 2, 2),strides = None))
        model.add(Conv3D(15,(3,3,3),data_format='channels_first',input_shape=self.InputShape,padding= 'same'))
        model.add(Dropout(0.1))
        #drop out of 10% after the first convolutional net
        model.add((Conv3D(30,(3,3,3),activation=self.Activation)))
        model.add(Dropout(.05))
        #drop out of 5% after the 2nd convolutional net
        self.Model = model
        #sets the model defined in build as the model

In [77]:
# Creates a Model based on the Class above with specified inputs/widths
NextModel = Conv_Sequential("NextModel_Revised",InputShape = (8, 200,200,200), Depth = 3,Widths = 4)
NextModel.Build()

In [78]:
#compiles using DLKIt
NextModel.Compile()

In [79]:
NextModel.Model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
activation_19 (Activation)   (None, 8, 200, 200, 200)  0         
_________________________________________________________________
dense_40 (Dense)             (None, 8, 200, 200, 4)    804       
_________________________________________________________________
dense_41 (Dense)             (None, 8, 200, 200, 4)    20        
_________________________________________________________________
dense_42 (Dense)             (None, 8, 200, 200, 4)    20        
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 4, 100, 100, 4)    0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 15, 100, 100, 4)   1635      
_________________________________________________________________
dropout_17 (Dropout)         (None, 15, 100, 100, 4)   0         
__________

In [80]:
#Save model using DLKIT
NextModel.Save()

In [7]:
!ls TrainedModels/

NextModel_Test1


In [115]:
#Example of Loading the Model Using DLKIt
LoadTest =ModelWrapper("NextModel_Test1") #insert name of model
LoadTest.Load()
print LoadTest.MetaData

{'Loss': 'mean_squared_logarithmic_error', 'Optimizer': None, 'Name': 'NextModel', 'InputMetaData': [{'Loss': 'mean_squared_logarithmic_error', 'Optimizer': None, 'Name': 'NextModel', 'InputMetaData': [], 'Widths': [10, 25, 50, 2], 'Activation': 'relu', 'InputShape': (200, 200, 200, 2), 'OutDir': 'TrainedModels/NextModel'}], 'Widths': [10, 25, 50, 2], 'Activation': 'relu', 'InputDir': 'TrainedModels/NextModel', 'InputShape': (200, 200, 200, 2), 'OutDir': 'TrainedModels/NextModel.1'}
